In [ ]:
import numpy as np
import sys 
import os 
import do_mpc 
import matplotlib.pyplot as plt
import pyomo.environ as pyomo
import numpy as np

<font color = 'orange'> Todo: Implement the "optimizing_battery_schedule_using_pyomo" solution in this Jupyter notebook, </br> see if I can mess around with the parameters to get it to work for my setting or not. Then see if I can include load shedding in some way - optimizing to minimize unserved kWh</font>

Objective = α * LoadSheddingObjective + β * ProfitObjective

<font color = 'orange'> Allow grid energy to be used to charge the battery?? Why not?? They do this in their optimization and it's a good idea </font> <br>
<font color = 'red'> Hmm but maybe not such a good idea because that places extra strain on the grid </font> <br>
But I can try it and then we can adjust later... </br>

Then optimize the capacity alongside scheduling in two stage optimization

In [2]:
class Battery: 
    def __init__(self, 
                 current_charge = 0.0,
                 capacity = 0.0,
                 charging_power_limit = 1.0,
                 discharging_power_limit = -1.0,
                 charging_efficiency = 0.90, 
                 discharging_efficiency = 0.95):
        self.current_charge = current_charge
        self.capacity = capacity
        self.charging_power_limit = charging_power_limit
        self.discharging_power_limit = discharging_power_limit
        self.charging_efficiency = charging_efficiency
        self.discharging_efficiency = discharging_efficiency
        

In [4]:
batt = Battery()
batt.current_charge

0.0

# Model 

In [ ]:


# create model
m = pyomo.ConcreteModel()

# Problem DATA
T = 24

Zmin = 0.0
Zmax = 2.0

Qmin = -1.0
Qmax = 1.0

# Generate prices, solar output and load signals
np.random.seed(42)
# P = np.random.rand(T)*5.0
# S = np.random.rand(T)
# L = np.random.rand(T)*2.0


# prob_false = 0.7
# prob_true = 0.3
# outage_hours = np.random.choice([False, True], size= T, p=[prob_false, prob_true])
net_load = np.random.rand(T) * 50.0 - np.random.rand(T) * 20.0
prices = np.random.rand(T)* 20.0 
feed_in_tariff = prices/2

# Indexes
times = range(T)
times_plus_1 = range(T+1)

# Parameters
m.net_load = pyomo.Param(times, initialize=lambda m, t: net_load[t])
m.prices = pyomo.Param(times, initialize=lambda m, t: prices[t])
#m.feed_in_tariff = pyomo.Param(times, initialize=lambda m, t: feed_in_tariff[t])

# Decisions variables
m.Q = pyomo.Var(times, domain=pyomo.Reals, bounds = (Qmin, Qmax))
m.Z = pyomo.Var(times_plus_1, domain=pyomo.NonNegativeReals, bounds = (Zmin, Zmax))
#m.Sell = pyomo.Var(times, domain=pyomo.NonNegativeReals)

# Objective
cost = sum(m.prices[t]*(m.net_load[t] - m.Q[t]) for t in times)
m.cost = pyomo.Objective(expr = cost, sense=pyomo.minimize)

# constraints
m.cons = pyomo.ConstraintList()
m.cons.add(m.Z[0] == 0.5*(Zmin + Zmax))

for t in times:
    # m.cons.add(pyomo.inequality(Qmin, m.Q[t], Qmax))
    # m.cons.add(pyomo.inequality(Zmin, m.Z[t], Zmax))
    m.cons.add(m.Z[t+1] == m.Z[t] - m.Q[t])
   
    
def charge_constraint_rule(m, t):
    if m.net_load[t] < 0:
        return m.net_load[t] - m.Q[t] <= 0
    else:
        return m.net_load[t] - m.Q[t] >= 0
m.charge_constraint = pyomo.Constraint(times, rule=charge_constraint_rule)

    

# solve
solver = pyomo.SolverFactory('glpk', executable = '/usr/local/bin/glpsol')

solver.solve(m)

# display results
print("Total cost =", m.cost(), ".")

for v in m.component_objects(pyomo.Var, active=True):
    print ("Variable component object",v)
    print ("Type of component object: ", str(type(v))[1:-1]) # Stripping <> for nbconvert
    varobject = getattr(m, str(v))
    print ("Type of object accessed via getattr: ", str(type(varobject))[1:-1])

    for index in varobject:
        print ("   ", index, varobject[index].value)

In [ ]:
import pyomo.environ as pyomo
import numpy as np

# create model
m = pyomo.ConcreteModel()

# Problem DATA
T = 24

Zmin = 0.0
Zmax = 50.0

Qmin = -25.0
Qmax = 25.0

# Generate prices, solar output and load signals
np.random.seed(42)
# P = np.random.rand(T)*5.0
# S = np.random.rand(T)
# L = np.random.rand(T)*2.0


# prob_false = 0.7
# prob_true = 0.3
# outage_hours = np.random.choice([False, True], size= T, p=[prob_false, prob_true])
net_load = np.random.rand(T) * 50.0 - np.random.rand(T) * 20.0
prices = np.random.rand(T)* 5.0
feed_in_tariff = np.random.rand(T)*0

# Indexes
times = range(T)
times_plus_1 = range(T+1)

# Parameters
m.net_load = pyomo.Param(times, initialize=lambda m, t: net_load[t]) # kW
m.prices = pyomo.Param(times, initialize=lambda m, t: prices[t]) # $/kWh
m.feed_in_tariff = pyomo.Param(times, initialize=lambda m, t: feed_in_tariff[t]) # $/kWh

# Decisions variables
m.Q = pyomo.Var(times, domain=pyomo.Reals, bounds = (Qmin, Qmax)) # kW
m.Z = pyomo.Var(times_plus_1, domain=pyomo.NonNegativeReals, bounds = (Zmin, Zmax)) # kWh
m.Sell = pyomo.Var(times, domain=pyomo.NonNegativeReals) # kW

# Objective
cost = sum(m.prices[t]*(m.net_load[t] - m.Q[t]) - (m.feed_in_tariff[t] * m.Sell[t]) for t in times)
m.cost = pyomo.Objective(expr = cost, sense=pyomo.minimize)

# Constraints
def initial_state_of_energy(m, t):
    return m.Z[0] == 0
m.initial_state_of_energy = pyomo.Constraint(times, rule=initial_state_of_energy)

def update_state_of_energy(m, t):
    return m.Z[t+1] == m.Z[t] - m.Q[t] # m.Q is positive when discharging and negative when charging 
m.update_state_of_energy = pyomo.Constraint(times, rule=update_state_of_energy)

def charge_constraint_rule(m, t):
    if m.net_load[t] < 0: # if net load is negative, we charge the battery as much as we can, and sell the rest to the grid 
        return m.net_load[t] - m.Q[t] + m.Sell[t] <= 0
    else:
        return m.net_load[t] - m.Q[t] >= 0 # if net load is positive, we can only discharge the battery [positive net load minus positive discharge]
m.charge_constraint = pyomo.Constraint(times, rule=charge_constraint_rule)

def when_discharge(m, t):
    if m.net_load[t] > 0:
        return m.Q[t] >= 0
    else:
        return m.Q[t] <= 0
m.when_discharge = pyomo.Constraint(times, rule=when_discharge)

def only_sell_excess(m, t):
    if m.net_load[t] > 0:
        return m.Sell[t] == 0
    else:
        return m.Sell[t] >= 0
m.only_sell_excess = pyomo.Constraint(times, rule=only_sell_excess)


# Solve
solver = pyomo.SolverFactory('gurobi', executable = '/usr/local/bin/gurobi_cl')

solver.solve(m)

#display results
print("Total cost =", m.cost(), ".")

for i in range(len(m.Q)):
    print(round(m.net_load[i], 1), pyomo.value(m.Z[i]), pyomo.value(m.Q[i]), round(pyomo.value(m.Sell[i]),1))

# for v in m.component_objects(pyomo.Var, active=True):
#     print ("Variable component object",v)
#     print ("Type of component object: ", str(type(v))[1:-1]) # Stripping <> for nbconvert
#     varobject = getattr(m, str(v))
#     print ("Type of object accessed via getattr: ", str(type(varobject))[1:-1])

#     for index in varobject:
#         print ("   ", index, varobject[index].value)
        
        

In [ ]:
import pyomo.environ as pyomo
import numpy as np

# create model
m = pyomo.ConcreteModel()

# Problem DATA
T = 24

Zmin = 0.0
Zmax = 2.0

Qmin = -1.0
Qmax = 1.0

# Generate prices, solar output and load signals
np.random.seed(42)
# P = np.random.rand(T)*5.0
# S = np.random.rand(T)
# L = np.random.rand(T)*2.0


# prob_false = 0.7
# prob_true = 0.3
# outage_hours = np.random.choice([False, True], size= T, p=[prob_false, prob_true])
net_load = np.random.rand(T) * 50.0 - np.random.rand(T) * 20.0
prices = np.random.rand(T)* 20.0 
feed_in_tariff = np.random.rand(T)* 5.0 

# Indexes
times = range(T)
times_plus_1 = range(T+1)

# Parameters
m.net_load = pyomo.Param(times, initialize=lambda m, t: net_load[t]) # kW
m.prices = pyomo.Param(times, initialize=lambda m, t: prices[t]) # $/kWh
m.feed_in_tariff = pyomo.Param(times, initialize=lambda m, t: feed_in_tariff[t]) # $/kWh

# Decisions variables
m.Q = pyomo.Var(times, domain=pyomo.Reals, bounds = (Qmin, Qmax)) # kW
m.Z = pyomo.Var(times_plus_1, domain=pyomo.NonNegativeReals, bounds = (Zmin, Zmax)) # kWh
m.Sell = pyomo.Var(times, domain=pyomo.NonNegativeReals) # kW

# Objective
cost = sum(m.prices[t]*(m.net_load[t] - m.Q[t]) - (m.feed_in_tariff[t] * m.Sell[t]) for t in times)
m.cost = pyomo.Objective(expr = cost, sense=pyomo.minimize)

# Constraints
def initial_state_of_energy(m, t):
    return m.Z[0] == 0
m.initial_state_of_energy = pyomo.Constraint(times, rule=initial_state_of_energy)

def update_state_of_energy(m, t):
    return m.Z[t+1] == m.Z[t] - m.Q[t] # m.Q is positive when discharging and negative when charging 
m.update_state_of_energy = pyomo.Constraint(times, rule=update_state_of_energy)

def charge_constraint_rule(m, t):
    if m.net_load[t] < 0: # if net load is negative, we can only charge the battery, and sell the rest to the grid 
        return m.net_load[t] - m.Q[t] + m.Sell[t] == 0
    else:
        return m.net_load[t] - m.Q[t] >= 0 # if net load is positive, we can only discharge the battery [positive net load minus positive discharge]
m.charge_constraint = pyomo.Constraint(times, rule=charge_constraint_rule)

def when_discharge(m, t):
    if m.net_load[t] > 0:
        return m.Q[t] >= 0
    else:
        return m.Q[t] <= 0
m.when_discharge = pyomo.Constraint(times, rule=when_discharge)

def only_sell_excess(m, t):
    if m.net_load[t] > 0:
        return m.Sell[t] == 0
    else:
        return m.Sell[t] >= 0
m.only_sell_excess = pyomo.Constraint(times, rule=only_sell_excess)


# Solve
solver = pyomo.SolverFactory('glpk', executable = '/usr/local/bin/glpsol')

solver.solve(m)

#display results
print("Total cost =", m.cost(), ".")

for i in range(len(m.Q)):
    print(round(m.net_load[i], 1), pyomo.value(m.Z[i]), pyomo.value(m.Q[i]), round(pyomo.value(m.Sell[i]),1))

# for v in m.component_objects(pyomo.Var, active=True):
#     print ("Variable component object",v)
#     print ("Type of component object: ", str(type(v))[1:-1]) # Stripping <> for nbconvert
#     varobject = getattr(m, str(v))
#     print ("Type of object accessed via getattr: ", str(type(varobject))[1:-1])

#     for index in varobject:
#         print ("   ", index, varobject[index].value)
        
        

In [ ]:
import pyomo.environ as pyomo 

# Original Data
outage_hours = [True, True, False, False, True, False, True, True, False, False, False, False, False, True]
net_load_profile = np.random.uniform(-50, 50, len(outage_hours))
time = range(len(outage_hours))
prices = np.random.uniform(0, 10, len(outage_hours))

unserved_load = np.array([net_load_profile[i] if outage_hours[i] else 0 for i in range(len(outage_hours))])

# Create model
m = ConcreteModel()

# Sets
m.time = Set(initialize = [t for t in time])
m.outage_hours = Param(m.time, initialize = outage_hours)
m.net_load_profile = Param(m.time, initialize = net_load_profile)
#m.prices = Param(m.time, initialize = prices)
m.unserved_load = Param(m.time, initialize = unserved_load, mutable = True)

# Static parameters
m.soe_initial = 0 # kWh (state of energy)
m.capacity = 100 # kWh
m.duration = 4 # hours
m.rating = m.capacity / m.duration # kW
m.charge_efficiency = 0.9
m.discharge_efficiency = 0.9

# Variables
m.charge = Var(m.time, domain = NonNegativeReals, bounds = (0, m.rating)) # energy charged to the battery system from the PV (kW)
m.discharge = Var(m.time, domain = NonNegativeReals, bounds = (0, m.rating)) # energy discharged from the battery system to the load (kW)
m.SOE = Var(m.time, domain = NonNegativeReals, bounds = (0, m.capacity)) # state of energy of the battery system (kWh)
#m.unserved_load = Var(m.time, domain = NonNegativeReals) # amount of unserved load (kW)

# Objective Function - Minimize unserved load 
# def obj_expression(m):
#     return sum(m.unserved_load[t] for t in m.time)
# m.obj = Objective(rule = obj_expression, sense = minimize) 

def obj_expression(m):
    return sum(m.prices[t] * (m.net_load_profile[t] - m.discharge[t]) for t in m.time)
m.obj = Objective(rule = obj_expression, sense = minimize) 

# Constraints 
# def unserved_load_rule(m, t): # to calculate the amount of unserved load at each time step
#     if net_load_profile[t] > 0:
#         if m.outage_hours[t] == True:
#             return m.unserved_load[t] == m.net_load_profile[t] - m.discharge[t]
#         else: # if no outage at that time step, load is served by grid
#             return m.unserved_load[t] == 0
#     else: # if net load is negative, there is no load to serve anyway 
#         return m.unserved_load[t] == 0
# m.unserved_load_rule = Constraint(m.time, rule=unserved_load_rule)



def soe_start_rule(m): # to set the initial state of energy stored at the battery system
    return m.SOE[0] == m.soe_initial
m.soe_start_rule = Constraint(rule=soe_start_rule)

def state_of_energy(m,t):  # to connect the states of the amount of energy stored at, energy discharged from, energy charged to, the battery system
    if t== 0:
        return m.SOE[t] == 0
    else:
        return m.SOE[t] == m.SOE[t-1] + m.charge[t-1] - m.discharge[t-1] 
m.state_of_energy_rule = Constraint(m.time, rule=state_of_energy)

def energy_flow_direction(m,t): # to ensure that the battery only discharges when net load is positive and charges when net load is negative 
    if m.net_load_profile[t] >= 0:
        return m.charge[t] == 0
    else:
        return m.discharge[t] == 0

def available_to_discharge(m, t): # to ensure that the amount of energy discharged from the battery system to the load is less than the amount of energy stored at the battery system
    return m.discharge[t] <= m.SOE[t]
m.available_to_discharge = Constraint(m.time, rule=available_to_discharge)

def demand_for_discharge(m, t): # to ensure that the amount of energy discharged from the battery system to the load is less than the amount of energy needed by the load
    return m.discharge[t] <= m.net_load_profile[t]
m.demand_for_discharge = Constraint(m.time, rule=demand_for_discharge)


def room_to_charge(m, t): # to ensure that the amount of energy charged to the battery system from the PV is less than the amount of energy stored at the battery system
    return m.charge[t] <= m.capacity - m.SOE[t]
m.room_to_charge = Constraint(m.time, rule=room_to_charge)

def available_to_charge(m, t): # to ensure that the amount of energy charged to the battery system from the PV is less than the amount of energy available from the PV
    #note: this is negative because battery charges when net load is negative 
    return m.charge[t] <= -m.net_load_profile[t]
m.available_to_charge = Constraint(m.time, rule=available_to_charge)



# Solve
solver = pyomo.SolverFactory('glpk', executable = '/usr/local/bin/glpsol')
solver.solve(m)
